In [36]:
!pip install requests beautifulsoup4 numpy

In [29]:
import xml.etree.ElementTree as ET
import requests
from bs4 import BeautifulSoup
import random
import numpy as np

In [ ]:
# Load and parse the main sitemap file
main_sitemap_path = 'sitemap.xml'
tree = ET.parse(main_sitemap_path)
root = tree.getroot()

# Namespace for parsing sitemap XML
namespace = {'ns': 'http://www.sitemaps.org/schemas/sitemap/0.9'}

In [19]:
# Initialize counters and lists
total_english_url_count = 0
total_french_url_count = 0
english_urls = []
french_urls = []

# Iterate through each sitemap in the main sitemap
for sitemap in root.findall('ns:sitemap', namespace):
    sitemap_url = sitemap.find('ns:loc', namespace).text
    response = requests.get(sitemap_url)

    # Parse the individual sitemap
    sitemap_tree = ET.fromstring(response.content)
    
    # Iterate over each URL in the sitemap
    for url_element in sitemap_tree.findall('ns:url', namespace):
        url = url_element.find('ns:loc', namespace).text

        # Check if English or French
        if "/en/" in url:
            total_english_url_count += 1
            english_urls.append(url)
        
        elif "/fr/" in url:
            total_french_url_count += 1
            french_urls.append(url)

print(f"Total English URLs: {total_english_url_count:,}")
print(f"Total French URLs: {total_french_url_count:,}")

Total English URLs: 251,372
Total French URLs: 252,305


In [21]:
with open('english_urls.txt', 'w') as file:
    for url in english_urls:
        file.write(url + '\n')

with open('french_urls.txt', 'w') as file:
    for url in french_urls:
        file.write(url + '\n')

In [25]:
sample_size = 100
iterations = 10
estimates = []

# Function to perform one sampling and size estimation
def estimate_size(sample_size, urls, total_url_count):
    sampled_urls = random.sample(urls, sample_size)
    total_sample_size_bytes = 0

    for url in sampled_urls:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        text_content = soup.get_text()
        page_size_bytes = len(text_content.encode('utf-8'))
        total_sample_size_bytes += page_size_bytes

    average_page_size_bytes = total_sample_size_bytes / sample_size
    estimated_total_size_bytes = average_page_size_bytes * total_url_count
    estimated_total_size_gb = estimated_total_size_bytes / (1024 ** 3)
    return estimated_total_size_gb

for i in range(iterations):
    estimate = estimate_size(sample_size, english_urls, total_english_url_count)
    estimates.append(estimate)
    print(f"Iteration {i+1}: Estimated size = {estimate:.2f} GB")

Iteration 1: Estimated size = 2.07 GB
Iteration 2: Estimated size = 3.64 GB
Iteration 3: Estimated size = 2.98 GB
Iteration 4: Estimated size = 3.48 GB
Iteration 5: Estimated size = 3.36 GB
Iteration 6: Estimated size = 1.90 GB
Iteration 7: Estimated size = 2.91 GB
Iteration 8: Estimated size = 2.62 GB
Iteration 9: Estimated size = 2.06 GB
Iteration 10: Estimated size = 4.24 GB

Mean Estimated Size: 2.93 GB
Standard Deviation: 0.73 GB


In [34]:
mean_estimate = np.mean(estimates)
std_deviation = np.std(estimates)

print(f"\nMean Estimated Size (of English pages only): {mean_estimate:.2f} GB")
print(f"Standard Deviation: {std_deviation:.2f} GB")


Mean Estimated Size (of English pages only): 2.93 GB
Standard Deviation: 0.73 GB


In [43]:
gb_to_bytes = 1_073_741_824  # Bytes in one gigabyte

# https://platform.openai.com/docs/guides/production-best-practices/managing-costs
words_to_tokens_ratio = 1000 / 750  # 1,000 tokens per 750 words

# https://platform.openai.com/docs/guides/embeddings/embedding-models
cost_per_thousand_tokens = 0.0004  # Cost per 1,000 tokens

# https://foolip.org/2008/05/17/how-much-a-thousand-words-are-worth/
average_bytes_per_word = 4.158

# Taking a conservative estimate: mean + 1 std, covering 84% of the data
size_in_gb = mean_estimate + std_deviation

size_in_bytes = size_in_gb * gb_to_bytes
total_words = size_in_bytes / average_bytes_per_word

total_tokens = total_words * words_to_tokens_ratio

total_cost = (total_tokens / 1000) * cost_per_thousand_tokens

print(f"Total tokens: {total_tokens:,.1f}")
print(f"Total Cost: ${total_cost:,.2f}")

Total tokens: 1,259,466,086.8
Total Cost: $503.79
